In [52]:
from bs4 import BeautifulSoup
import requests
import os
import re

In [24]:
base_url = r'https://www.icpsr.umich.edu/files/NACJD/ORIs/'
response = requests.get(base_url)

In [25]:
soup = BeautifulSoup(response.text, 'html.parser')

# list of links to pages with counties, county law enforcemen agencies and their ORIs
links = [os.path.join(base_url, link.get('href')) for link in soup.find_all('a')][4:-2]

'https://www.icpsr.umich.edu/files/NACJD/ORIs/01oris.html'

In [ ]:
def find_county_codes(text):
    
    county_dict = {}
    county_raw, code_raw = text.a.get('name').split('_(FIPS=')
    county = ' '.join(county_raw.split('_')[1:])
    code = [i.strip(')') for i in re.split('=|_UCR', code_raw) if i]
    county_dict['county'] = county
    county_dict['fips'] = code[0]
    county_dict['ucr'] = code[1]
    
    return county_dict

In [ ]:
def find_agency_ori(text):
    
    agency_raw = text.text.split('ORI9')[1].strip().split('\n')
    agency_ori = []
    for element in agency_raw:
        data = [i for i in element.strip().split('   ') if i]
        agency_dict = {}
        agency_dict['agency'] = data[0]
        agency_dict['ori7'] = data[1]
        agency_dict['ori9'] = data[2]
        agency_dict['state_abbr'] = data[2][:2]
        agency_ori.append(agency_dict)
    
    return agency_ori


In [134]:
def scrape_by_state(link):

    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')

    agencies = []
    for s, a in zip(soup.find_all('h3'), soup.find_all('pre')):
        county = find_county_codes(s)
        ori_codes = find_agency_ori(a)
        for agency in ori_codes:
            agencies.append({**county, **agency})


    return agencies   

In [ ]:
agencies = []
for link in links:
    agencies.extend(scrape_by_state(link))
    

In [138]:
len(agencies)

21993

In [139]:
agencies[10000]

{'county': 'COOSA',
 'fips': '037',
 'ucr': '019',
 'agency': "COOSA COUNTY SHERIFF'S OFFICE",
 'ori7': '  AL02200',
 'ori9': 'AL0220000',
 'state_abbr': 'AL'}